In [1]:
import pandas as pd
import spacy
from tqdm import tqdm

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx
import json

import matplotlib.pyplot as plt

2023-03-08 22:02:02.991856: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 22:02:03.968921: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-08 22:02:03.968951: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-08 22:02:05.934782: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Análise dos dados news.tsv
- Quantidade de linhas com entities de title e abstract nulas

In [20]:
newsFile = "dataset/news.tsv"
newsDf = pd.read_csv(newsFile, header=None, sep="\t", skiprows = 0, nrows=10, names=[
                  'id', 'category', 'subcategory', 'title', 'abstract', 'url',
                  'title_entities', 'abstract_entities'
              ])
newsDf = newsDf.fillna("")

In [30]:
newsDf

,id,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
5,N75236,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
6,N99744,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."
7,N5771,health,cardio,Check Houston traffic map for current road con...,,https://assets.msn.com/labs/mind/AAEKnO1.html,"[{""Label"": ""Houston"", ""Type"": ""G"", ""WikidataId...",[]
8,N124534,sports,football_nfl,Should NFL be able to fine players for critici...,Several fines came down against NFL players fo...,https://assets.msn.com/labs/mind/AAJ4lap.html,"[{""Label"": ""National Football League"", ""Type"":...","[{""Label"": ""National Football League"", ""Type"":..."
9,N51947,news,newsscienceandtechnology,"How to record your screen on Windows, macOS, i...",The easiest way to record what's happening on ...,https://assets.msn.com/labs/mind/AADlomf.html,"[{""Label"": ""Microsoft Windows"", ""Type"": ""J"", ""...",[]


## Análise dos dados behaviors.tsv

In [22]:
behavFile = "dataset/behaviors.tsv"
behavDf = pd.read_csv(behavFile, header=None, sep="\t", skiprows = 0, nrows=10, names=[
                  'impression_id', 'user_id', 'time', 'history', 'impressions'
              ])
behavDf = behavDf.fillna("")

In [25]:
behavDf

,impression_id,user_id,time,history,impressions
0,1,U87243,11/10/2019 11:30:54 AM,N8668 N39081 N65259 N79529 N73408 N43615 N2937...,N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...
1,2,U598644,11/12/2019 1:45:29 PM,N56056 N8726 N70353 N67998 N83823 N111108 N107...,N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...
2,3,U532401,11/13/2019 11:23:03 AM,N128643 N87446 N122948 N9375 N82348 N129412 N5...,N103852-0 N53474-0 N127836-0 N47925-1
3,4,U593596,11/12/2019 12:24:09 PM,N31043 N39592 N4104 N8223 N114581 N92747 N1207...,N38902-0 N76434-0 N71593-0 N100073-0 N108736-0...
4,5,U239687,11/14/2019 8:03:01 PM,N65250 N122359 N71723 N53796 N41663 N41484 N11...,N76209-0 N48841-0 N67937-0 N62235-0 N6307-0 N3...
5,6,U521853,11/11/2019 10:47:31 AM,N8668 N29136 N128643 N9740 N9375 N52911 N12090...,N32154-0 N67747-0 N47257-0 N98178-1
6,7,U687515,11/13/2019 5:31:06 AM,N91810 N96438 N104027 N53650,N12681-0 N10802-0 N93856-0 N96729-0 N46415-0 N...
7,8,U192112,11/11/2019 9:06:04 AM,N9740 N59820 N18389 N23320 N12322 N9375 N11563...,N98178-1 N32154-0
8,9,U530668,11/11/2019 2:10:15 PM,N67770 N65823 N35599 N8753 N126368 N32221 N844...,N49168-0 N123077-1 N55792-0 N47257-0
9,10,U290933,11/10/2019 11:54:34 AM,N14678 N71340 N65259 N92085 N31043 N70385 N123...,N92691-0 N10654-0 N107542-0 N14509-0 N63262-0 ...


## Processamento
- Processar entidades
    - Cada entidade com Label, Type, WikiDataId
    - Concatenar entidades de title e abstract, remover duplicatas
- Iterar notícias:
    - Criar nodo da notícia apenas se existir pelo menos 1 entidade
    - Adicionar aresta "related" entre todas os nodos do tipo entidade de uma mesma notícia
    - Adicionar aresta "content" entre a notícia e cada um de seus nodos do tipo entidade 
- Iterar usuários:
    - Criar nodo do usuário
    - Adicionar aresta "reading" entre nodo do usuário e cada um dos nodos das notícias que ele leu (se existir)
    
    
- Sistema relação entre objetos: se já existir uma relação entre 2 entidades, somar um contador na aresta
- Cada nodo com nome e tipo (notícia, usuário ou entity)
    - Notícia com campo data e flag "new"


In [4]:
from neo4j import GraphDatabase


# WRITE

@staticmethod
def createEntity(tx, typee, label, WDId):
    result = tx.run("MERGE (a:Entity{label: $label, typee: $typee, WDId: $WDId}) ", label=label, typee=typee, WDId=WDId)
    return

@staticmethod
def createNews(tx, ID, category, subcategory, title, abstract, url, new):
    result = tx.run("MERGE (a:News{id: $ID, title: $title, abstract: $abstract, category: $category, subcategory: $subcategory, url: $url, new: $new}) ", ID=ID, title=title, abstract=abstract, category=category, subcategory=subcategory, url=url, new=new)
    return

@staticmethod
def createUser(tx, ID, new): #new/last acess
    result = tx.run("MERGE (a:User{ID: $ID, new=$new}) ", ID=ID, new=new)
    return

@staticmethod
def connectUserToNews(tx, userID, newsID, time):
    result = tx.run("MATCH (u:User), (n:News) "
                    "WHERE u.id=$userID AND n.id=$newsID "
                    "MERGE (u)-[rd1:READ]->(n) "
                    "MERGE (n)-[rd2:READ]>(u) "
                    "SET rd1.time=$time "
                    "SET rd2.time=$time ", userID=userID, newsID=newsID, time=time)
    return

@staticmethod
def connectEntityToEntity(tx, entity1Label, entity2Label):
    result = tx.run("MATCH (e1:Entity), (e2:Entity) "
                    "WHERE e1.label=$entity1Label AND e2.label=$entity2Label "
                    "MERGE (e1)-[rel1:RELATED]->(e2) "
                    "MERGE (e2)-[rel2:RELATED]->(e1) "
                    "SET rel1.nCon=1 "
                    "SET rel2.nCon=1 ", entity1Label=entity1Label, entity2Label=entity2Label)
    return

@staticmethod
def connectNewsToEntity(tx, newsID, entityLabel):
    result = tx.run("MATCH (n:News), (e:Entity) "
                    "WHERE n.id=$newsID AND e.label=$entityLabel "
                    "MERGE (n)-[con1:CONTENT]->(e) "
                    "MERGE (e)-[con2:CONTENT]->(n) ", newsID=newsID, entityLabel=entityLabel)
    return


@staticmethod
def updateNConEntToEnt(tx, entity1Label, entity2Label):
    result = tx.run("MATCH (e1:Entity {label: $entity1Label}), (e2:Entity {label: $entity2Label}) " 
                    "MATCH (e1)-[rel1:RELATED]->(e2) "
                    "MATCH (e2)-[rel2:RELATED]->(e1) "
                    "SET rel1.nCon=rel1.nCon+1 "
                    "SET rel2.nCon=rel2.nCon+1 ", entity1Label=entity1Label, entity2Label=entity2Label)
    return 

# READ

@staticmethod
def entToEntConExists(tx, entity1Label, entity2Label):
    result = tx.run("MATCH (e1:Entity {label: $entity1Label}), (e2:Entity {label: $entity2Label}) " 
                    "MATCH (e1)-[rel:RELATED]->(e2) "
                    "WITH COUNT(rel) > 0  as node_exists "
                    "RETURN node_exists ", entity1Label=entity1Label, entity2Label=entity2Label)
    return result.fetch(1)[0][0]


@staticmethod
def newsExists(tx, newsID):
    result = tx.run("MATCH (n:News {label: $newsID}) " 
                    "WITH COUNT(n) > 0  as node_exists "
                    "RETURN node_exists ", newsID=newsID)
    return result.fetch(1)[0][0]



In [29]:
uri="bolt://localhost:7687"
user="neo4j"
password="88888888"

driver = GraphDatabase.driver(uri, auth=(user, password))

with driver.session() as session:
    for index, row in newsDf.iterrows():

        titleEntities = json.loads(row["title_entities"])
        abstractEntities = json.loads(row["abstract_entities"])
        entitiesLabel = []
        entities = []
        for ent in titleEntities:
            if ent['Label'] not in entitiesLabel:
                entities.append(ent)
                entitiesLabel.append(ent['Label'])
        for ent in abstractEntities:
            if ent['Label'] not in entitiesLabel:
                entities.append(ent)
                entitiesLabel.append(ent['Label'])

        if(entitiesLabel == []):
            continue
        
        # Insert News
        print(row['id'] + " ---> " + row['title'])
        session.execute_write(createNews, row['id'], row['category'], row['subcategory'], row['title'], row['abstract'], row['url'], '0')
        
        # Insert Entities
        for ent in entities:
            session.execute_write(createEntity, ent['Type'], ent['Label'], ent['WikidataId'])
            session.execute_write(connectNewsToEntity, row['id'], ent['Label'])
        
        count = 1
        for ent1 in entities:
            for ent2 in entities[count:]:
                if ent1['Label'] != ent2['Label']:
                    exists = session.execute_read(entToEntConExists, ent1['Label'], ent2['Label'])
                    if(exists):
                        session.execute_write(updateNConEntToEnt, ent1['Label'], ent2['Label'])
                    else:
                        session.execute_write(connectEntityToEntity, ent1['Label'], ent2['Label'])
            count+=1
                        
    for index, row in behavDf.iterrows():
        newsList = []
            
        newsToParse = row['impressions'].split(" ")
        for news in newsToParse:
            splitedNewsFormat = news.split("-")
            newsID = splitedNewsFormat[0]
            newsClicked = splitedNewsFormat[1]
            if newsClicked == "1":
                newsList.append(newsID)

       
        if len(newsList)>0:
            userCreated = False
            time = row['time']
            
            for newsID in newsList:
                exists = session.execute_read(newsExists, newsID)
                if not exists:
                    continue
                if not userCreated:
                    print(row['user_id'])
                    session.execute_write(createUser, row['user_id'], '0')
                    userCreated = True
                session.execute_write(connectUserToNews, row['user_id'], newsID, time)
        
        

        
driver.close()

N88753 ---> The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By
N45436 ---> Walmart Slashes Prices on Last-Generation iPads
N23144 ---> 50 Worst Habits For Belly Fat
N86255 ---> Dispose of unwanted prescription drugs during the DEA's Take Back Day
N93187 ---> The Cost of Trump's Aid Freeze in the Trenches of Ukraine's War
N75236 ---> I Was An NBA Wife. Here's How It Affected My Mental Health.
N99744 ---> How to Get Rid of Skin Tags, According to a Dermatologist
N5771 ---> Check Houston traffic map for current road conditions
N124534 ---> Should NFL be able to fine players for criticizing officiating?
N51947 ---> How to record your screen on Windows, macOS, iOS or Android
